In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Importing the required libs

In [7]:
from google.adk.agents import Agent,ParallelAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool,google_search
from google.genai import types

#### Gemini API Key

In [5]:
import os
from  kaggle_secrets import UserSecretsClient
key_label = "GOOGLE_API_KEY"
try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret(key_label)
    os.environ[key_label] = GOOGLE_API_KEY
    print(f"Gemini API key set up completed !!")
except Exception as e:
    print(f"Unable to find the {key_label} in the kagggle secrets. Got exception {e}")

Gemini API key set up completed !!


#### LLM Retry options

In [9]:
retry_config = types.HttpRetryOptions(
    attempts = 5,
    exp_base =7,
    initial_delay=1,
    http_status_codes=[429,500,503,504]
)

### Six Thinking Hats(Agents)

#### Approach 1 
Agents as tools without a fixed parallel workflow

#### WhiteHatAgent

In [15]:
white_hat_instructions="""You are a specialized agent which helps to give a fact based , unbiased information about the problem you are asked about.
Use the google_search tool to get the facts and data about the problem. Analyze only based on the facts and data.
DO NOT bring any emotional,critical or creative perspectives.
"""

In [48]:
white_hat_agent = Agent(
    name = "WhiteHatAgent",
    model = Gemini(
        model = "gemini-2.5-flash-lite",
        retry_options= retry_config
    ),
    instruction=white_hat_instructions,
    tools=[google_search],
    output_key="white_hat_agent_findings"
)

#### RedHatAgent


In [20]:
red_hat_instructions = """ You are a specialized agent which analyzes the problem from the emotional angle.
Your perspective is more about the feelings and emotions. DO NOT provide any analysis from the facts nor do any critical analysis .
"""

In [47]:
red_hat_agent = Agent(
    name ="RedHatAgent",
    model = Gemini(
        model = "gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction=red_hat_instructions,
    output_key="red_hat_agent_findings"
)

#### GreenHatAgent

In [22]:
green_hat_instructions = """ You are a specialized agent which thinking of very creative , out of the box ideas for the problem given to you.
You analysis includes new ideas without worrying about the feasibility . DO NOT think from emotional or critical aspects .
"""

In [44]:
green_hat_agent = Agent(
    name = "GreenHatAgent",
    model = Gemini(
        model  = "gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction=green_hat_instructions,
    output_key="green_hat_agent_findings"
)

#### YellowHatAgent

In [27]:
yellow_hat_instructions = """You are a specialized agent which looks at the pros of the solution , the postive aspects .
Your analysis of the solution or idea is to look out for the benefits and the positive aspects ONLY. 
DO NOT asses the cons """

In [45]:
yellow_hat_agent = Agent(
    name = "YellowHatAgent",
    model = Gemini(
        model = "gemini-2.5-flash-lite",
        retry_options = retry_config
    ),
    instruction = yellow_hat_instructions,
    output_key = "yellow_hat_agent_findings"
)

#### BlackHatAgent

In [29]:
black_hat_instructions = """ You are a specialized agent whose purpose is to provide the most critical analysis of a solution or idea.
You look only about the cons and the problems associated with a solution or idea. DO NOT point any positive aspects of the solution or idea.
"""

In [46]:
black_hat_agent = Agent(
    name = "BlackHatAgent",
    model = Gemini(
        model  = "gemini-2.5-flash-lite",
        retry_options = retry_config
    ),
    instruction = black_hat_instructions,
    output_key = "black_hat_agent_findings"
)

#### supervisor agent

In [32]:
supervisor_instructions = """
You are a creative problem solving agent . Given a problem you take help of the specialized  agent tools
that help you gain insights about the problem from different angles. Using the different perspectives gained from the 
helper agents you give a more creative  solution to the problem at hand.
The helper agents tools are 
1. `WhiteHatAgent` helps you with bare bones facts and data about the problem 
2. `RedHatAgent` helps you with the emotional angle of the problem 
3. `GreenHatAgent` gives you very out of the box ideas 
4. `YellowHatAgent` gives the more positive and optimistic ideas 
5. `BlackHatAgent` is more critical and points out the flaws in the ideas
"""

In [49]:
creative_problem_solver = Agent(
    name="CreativeProblemSolverAgent",
    model= Gemini(
        model = "gemini-2.5-flash-lite",
        retry_options = retry_config
    ),
    instruction = supervisor_instructions,
    tools = [AgentTool(white_hat_agent),AgentTool(red_hat_agent),AgentTool(green_hat_agent),
            AgentTool(yellow_hat_agent),AgentTool(black_hat_agent)]
)

#### Runner 

In [50]:
runner  = InMemoryRunner(agent=creative_problem_solver)
response = await runner.run_debug("""
I am working software engineer . I have to work 8 hours on each week day . 
I also go to GYM for 1 hour after work on each work day . I am planning to change my job to data science .
So I have to apply for jobs and prepare for the interview process . At the same time I am wanting to 
add more productive leisure in my life so that I don't end up wasting time online . I also want to improve
my social life . Help me figure out if learning guitar for twice a week 1 hour classes is a good choice or not ? Assume this is sufficient information 
""")


 ### Created new session: debug_session_id

User > 
I am working software engineer . I have to work 8 hours on each week day . 
I also go to GYM for 1 hour after work on each work day . I am planning to change my job to data science .
So I have to apply for jobs and prepare for the interview process . At the same time I am wanting to 
add more productive leisure in my life so that I don't end up wasting time online . I also want to improve
my social life . Help me figure out if learning guitar for twice a week 1 hour classes is a good choice or not ? Assume this is sufficient information 



ClientError: 404 NOT_FOUND. {'error': {'code': 404, 'message': 'models/gemma-3-12b is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.', 'status': 'NOT_FOUND'}}

In [42]:
response

[Event(model_version='gemini-2.5-flash-lite', content=Content(
   parts=[
     Part(
       text="""I will start by gathering information about the time commitment for job applications and interview preparation.
 Then, I will consult the WhiteHatAgent to understand the typical time demands of transitioning into a data science role, considering job searching and interview preparation.
 Concurrently, I will use the RedHatAgent to explore the emotional and social aspects of the user's current routine and their desire for improved social life.
 I will also engage the GreenHatAgent to brainstorm unconventional ways to integrate guitar learning with other goals, possibly finding overlaps or efficiencies.
 Next, the YellowHatAgent will be used to highlight the potential positive impacts of learning guitar on the user's well-being, productivity, and social life.
 Finally, I will use the BlackHatAgent to identify potential drawbacks or challenges of adding guitar classes to the user's schedule,

#### Approach 2 
Fixed parallel workflow with a main supervisor agent 